На этой неделе вам предстоит попробовать добавить в вашу регрессионную модель дополнительные признаки.

Во-первых, для прогнозирования можно использовать информацию, содержащуюся в сырых данных:

 * средняя длительность поездок
 * среднее количество пассажиров
 * среднее расстояние по счётчику
 * доли географических зон, в которые совершаются поездки
 * доли поездок, совершаемых по тарифам каждого из типов
 * доли способов оплаты поездок
 * средняя стоимость поездок
 * доли провайдеров данных
 
Все эти признаки можно использовать только с задержкой, то есть, при прогнозировании y^T+i|T эти признаки должны быть рассчитаны по данным не позднее момента времени T. Каждый из этих признаков можно использовать по-разному: как сырые значения за последние несколько часов, так и средние за последний день, неделю, месяц и т. д.

Во-вторых, чтобы улучшить качество прогнозов в аномальные периоды, вы можете найти информацию о потенциально влияющих на количество поездок событиях, таких, как государственные праздники. Проанализируйте, как именно поведение пассажиров меняется во время этих событий, и создайте признаки, отражающие эти изменения. Как показывает наш опыт, правильный учёт праздничных дней часто позволяет существенно уменьшить среднюю ошибку прогноза.

В-третьих, можно использовать признаки, связанные с географией. Например, скорее всего, суммарное количество поездок, совершаемых из географической зоны, пропорционально площади этой зоны. Для зон, прилегающих к аэропорту, может быть характерен специфический паттерн дневной сезонности, связанный с тем, что спрос на такси будет повышаться в те часы, когда общественный транспорт перестаёт работать. В деловом центре максимальное количество поездок будет приходиться на начало и окончание рабочего дня, на Бродвее — на время начала и окончания спектаклей. Все эти идеи не обязательно верны, мы приводим их здесь только для того, чтобы продемонстрировать принцип рассуждений. Ещё один пример географического признака: можно попробовать добавить идентификатор боро, который можно найти в файле https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv. Кроме того, нам кажется перспективным использование в качестве фактора количества поездок, совершённых за прошлый час/день и т. д. из соседних географических зон, или количества поездок, совершённых за прошлый час/день в текущую географическую зону.

Много примеров других признаков, которые можно использовать при регрессионном прогнозировании, можно найти в лекции Вадима Стрижова.

In [1]:
import warnings
import math
import os

import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import  LinearRegression
from xgboost import XGBRegressor
from datetime import datetime
from tqdm import tqdm 

warnings.filterwarnings('ignore')
PATH_TO_DATA = 'data'
PATH_TO_RAW_DATA = '../data'

In [2]:
def get_features(df, K = 20):
    length = df.shape[0] + 1
    features = pd.DataFrame()
    for i in range(1,K+1):
        features['sin_%d' % i] = [ np.sin(t  * 2. * i * np.pi / 168.) for t in np.arange(1, length)]
        features['cos_%d' % i] = [ np.cos(t  * 2. * i * np.pi / 168.) for t in np.arange(1, length)]

    features[['h%d' % i for i in range(0, 24)]] = pd.get_dummies(df.index.hour)
    features[['dw%d' % i for i in range(0, 7)]] = pd.get_dummies(df.index.weekday)
    features[['dm%d' % i for i in range(0, 31)]] = pd.get_dummies(df.index.day)
    features['is_weekend'] = (df.index.weekday > 4).astype(float)
    
    return features

In [3]:
def rolling_window(a, window):
    a = np.concatenate((np.zeros(window), a), axis=0)
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)[:-1]

1\. Загрузите обучающие выборки прошлой недели, перечислите используемые в моделях признаки и посчитайте Qmay — качество прогнозов моделей, настроенных на данных до апреля 2016, в мае 2016.

In [4]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train.csv'), index_col=0, parse_dates=[0])
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_05.csv'), index_col=0, parse_dates=[0])
kaggle_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_06.csv'), index_col=0, parse_dates=[0])


concat_df = pd.concat((train_df, test_df, kaggle_df))
print(concat_df.shape)
concat_df.head()

(4368, 102)


,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
2016-01-01 00:00:00,80.0,144.0,50.0,77.0,319.0,402.0,531.0,617.0,846.0,267.0,...,12.0,0.0,2.0,44.0,5.0,41.0,4.0,70.0,7.0,66.0
2016-01-01 01:00:00,91.0,211.0,49.0,134.0,404.0,420.0,370.0,453.0,594.0,224.0,...,29.0,0.0,5.0,2.0,2.0,4.0,0.0,47.0,1.0,29.0
2016-01-01 02:00:00,90.0,146.0,23.0,110.0,393.0,425.0,313.0,366.0,377.0,138.0,...,47.0,0.0,3.0,0.0,4.0,0.0,0.0,69.0,1.0,14.0
2016-01-01 03:00:00,32.0,87.0,16.0,62.0,252.0,399.0,324.0,309.0,327.0,166.0,...,46.0,0.0,2.0,4.0,5.0,1.0,0.0,21.0,0.0,9.0
2016-01-01 04:00:00,24.0,43.0,10.0,53.0,145.0,254.0,264.0,333.0,318.0,145.0,...,43.0,0.0,0.0,1.0,1.0,0.0,0.0,26.0,1.0,6.0


In [5]:
destination_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'small_destination.csv'), index_col=0, parse_dates=[0])
print(destination_df.shape)
destination_df.head()

(4368, 102)


,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
2016-01-01 00:00:00,74.0,154.0,53.0,74.0,204.0,275.0,274.0,306.0,442.0,152.0,...,34.0,0.0,5.0,2.0,3.0,1.0,0.0,3.0,0.0,1.0
2016-01-01 01:00:00,107.0,218.0,66.0,119.0,336.0,282.0,339.0,454.0,563.0,200.0,...,52.0,0.0,16.0,4.0,1.0,4.0,0.0,3.0,0.0,2.0
2016-01-01 02:00:00,93.0,194.0,37.0,103.0,333.0,216.0,260.0,341.0,370.0,125.0,...,59.0,0.0,15.0,3.0,12.0,2.0,0.0,4.0,2.0,1.0
2016-01-01 03:00:00,72.0,183.0,36.0,88.0,278.0,149.0,209.0,240.0,294.0,116.0,...,75.0,0.0,13.0,24.0,10.0,6.0,1.0,10.0,3.0,9.0
2016-01-01 04:00:00,76.0,127.0,23.0,55.0,171.0,122.0,160.0,197.0,201.0,70.0,...,61.0,0.0,14.0,52.0,17.0,14.0,1.0,15.0,7.0,13.0


2\. Попробуйте добавить признаки. Используйте идеи, которые мы предложили, или какие-то свои. Обучайте обновлённые модели на данных до апреля 2016 включительно и считайте качество новых прогнозов на мае. Удаётся ли вам улучшить качество? Не нужно ли увеличить сложность регрессионной модели? Если добавляемый признак не улучшает качество, всё равно оставьте доказательства этому в ноутбуке, чтобы ваши коллеги это видели при проверке.

In [6]:
feature_df = get_features(concat_df)
print(feature_df.shape)
feature_df.head()

(4368, 103)


,sin_1,cos_1,sin_2,cos_2,sin_3,cos_3,sin_4,cos_4,sin_5,cos_5,...,dm22,dm23,dm24,dm25,dm26,dm27,dm28,dm29,dm30,is_weekend
0,0.037391,0.999301,0.074730,0.997204,0.111964,0.993712,0.149042,0.988831,0.185912,0.982566,...,0,0,0,0,0,0,0,0,0,0.0
1,0.074730,0.997204,0.149042,0.988831,0.222521,0.974928,0.294755,0.955573,0.365341,0.930874,...,0,0,0,0,0,0,0,0,0,0.0
2,0.111964,0.993712,0.222521,0.974928,0.330279,0.943883,0.433884,0.900969,0.532032,0.846724,...,0,0,0,0,0,0,0,0,0,0.0
3,0.149042,0.988831,0.294755,0.955573,0.433884,0.900969,0.563320,0.826239,0.680173,0.733052,...,0,0,0,0,0,0,0,0,0,0.0
4,0.185912,0.982566,0.365341,0.930874,0.532032,0.846724,0.680173,0.733052,0.804598,0.593820,...,0,0,0,0,0,0,0,0,0,0.0


In [7]:
values = [rolling_window(concat_df[c], 5) for c in train_df.columns]
previous_rodes = pd.Series(values, index=train_df.columns)

values = [rolling_window(destination_df[c], 5) for c in train_df.columns]
destination_rodes = pd.Series(values, index=train_df.columns)

In [8]:
submition_df = pd.DataFrame()

train_size = train_df.shape[0]
kaggle_size = kaggle_df.shape[0]

models = [LinearRegression, RandomForestRegressor, XGBRegressor]
score = pd.Series()
best_score = math.inf
size_of_predictions = 6 * 739 * test_df.shape[1]

In [9]:
for m in models:
    test_prediction_df = pd.DataFrame()
    kaggle_prediction_df = pd.DataFrame()
    
    for c in tqdm(train_df.columns):
        X = np.concatenate((feature_df[:train_size].values, previous_rodes[c][:train_size], destination_rodes[c][:train_size]), 
                           axis=1)
        y = train_df[c]

        X_test = np.concatenate((feature_df[train_size:-kaggle_size].values, previous_rodes[c][train_size:-kaggle_size], 
                                 destination_rodes[c][train_size:-kaggle_size]), axis=1)
        X_kaggle = np.concatenate((feature_df[-kaggle_size + 7:].values, previous_rodes[c][-kaggle_size  + 7:], 
                                  destination_rodes[c][-kaggle_size  + 7:]), axis=1)

        model = m().fit(X, y)

        test_prediction_df[c] = model.predict(X_test)
        kaggle_prediction_df[c] = model.predict(X_kaggle)
        
        
    error = 0

    for c in test_prediction_df.columns:
        error += sum(sum(abs(rolling_window(test_prediction_df[c], 6) - rolling_window(test_df[c], 6)))) / size_of_predictions

    score[m.__name__] = error

    if error < best_score:
        submition_df = kaggle_prediction_df

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 102/102 [00:00<00:00, 1273.80it/s]


3\. Когда вы примете решение остановиться и перестать добавлять признаки, постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза по следующему функционалу:

$$Q_{june} = {1 \over R * 715 * 6} +\sum_{r=1}^{R} \sum_{2016.04.30\ 23:00}^{2016.05.31\ 17:00}\sum_{i=1}^{6} |\hat{y}_{T|T+i} - y^r_{T+i}|$$

Убедитесь, что среднее качество прогнозов увеличилось.

In [10]:
score.sort_values()

XGBRegressor             15.770454
RandomForestRegressor    15.818759
LinearRegression         16.948431
dtype: float64

5\. Переобучите итоговые модели на данных до мая 2016 включительно, постройте прогнозы на июнь для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в уже знакомом вам формате: geoID, histEndDay, histEndHour, step, y

In [11]:
index = []
values = []

start_period = june_start_period = datetime(2016, 5, 31, 17, 0, 0)
dates = [start_period + pd.DateOffset(hours=i) for i in range(721)]

for c in tqdm(submition_df.columns):
    t = rolling_window(submition_df[c], 6)
    for idx, x in enumerate(t):
        for off, i in enumerate(x):
            tm = dates[idx + off]
            index.append('{}_2016-{:02}-{:02}_{}_{}'.format(c, tm.month, tm.day, tm.hour, 6 - off))
            values.append(int(i))
            
result_df = pd.DataFrame({'y': values}, index=index)
result_df.index.name = 'id'
result_df.to_csv(os.path.join(PATH_TO_DATA, 'result.csv'))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 102/102 [00:01<00:00, 71.93it/s]


6\. Загрузите полученный файл на kaggle: https://inclass.kaggle.com/c/yellowtaxi. Добавьте в ноутбук ссылку на сабмишн.